## Laboratorium 4
#### Bartosz Hanc

In [32]:
"""Requirements:
numpy==1.23.5
scikit-learn==1.2.1
scipy==1.10.0
kmedoids==0.4.3
"""

import numpy as np


1. Zaimplementuj przynajmniej 3 "metryki" spośród wymienionych: cosinusowa, LCS,
   DICE, euklidesowa, Levenshteina.

Ponieważ do klasteryzacji użyłem algorytmu PAM k-medoids
(https://en.wikipedia.org/wiki/K-medoids) dostępnego w bibliotece `kmedoids`,
którego implementacja zawiera krok przypisania punktu do klastra wyznaczonego
przez *najbliższy* medoid, więc wykorzystana funkcja podobieństwa wyrazów
powinna mieć własności standardowej metryki tj. $d(x,y) = 0\iff x=y$, $\forall
x,y: d(x,y) \geq 0$ i odpowiadać intuicyjnemu postrzeganiu, iż jeśli $d(x,y) <
d(x,z)$ to $y$ jest bliżej $x$, czyli wyraz $y$ jest bardziej podobny do $x$ niż
wyraz $z$. Spośród wymienionych funkcji jedynie odległość euklidesowa i
Levenshteina spełniają te kryteria. Z tego względu jako trzecią metrykę wybrałem
metrykę taksówkową spoza listy.

In [2]:
from sklearn.metrics import pairwise_distances as d

# Use cases:
# d(X, Y, metric="euclidean")
# d(X, Y, metric="manhattan")
# d(X, Y, metric="hamming") # This one is very slow because it does not support
# sparse matrices (and our matrix has ~1e8 elements)


2. Zaimplementuj przynajmniej 1 sposoby oceny jakości klasteryzacji (np. indeks
   Daviesa-Bouldina).

In [24]:
from sklearn.metrics import davies_bouldin_score as DB_idx


3. Stwórz stoplistę najczęściej występujących słów i zastosuj ją jako
   pre-processing dla nazw. Algorytmy klasteryzacji powinny działać na dwóch
   wariantach: z pre-processingiem i bez pre-processingu.

In [4]:
docs = np.array(open("lines.txt", "r").readlines())

4. Wykonaj klasteryzację zawartości załączonego pliku (lines.txt) przy użyciu
   metryk zaimplementowanych w pkt. 1. Każda linia to adres pocztowy firmy,
   różne sposoby zapisu tego samego adresu powinny się znaleźć w jednym
   klastrze.

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
import kmedoids

freq_vecs = CountVectorizer().fit_transform(docs)

k = 1000
c_euclid = kmedoids.fasterpam(d(freq_vecs, freq_vecs, metric="euclidean"), k)
c_manhat = kmedoids.fasterpam(d(freq_vecs, freq_vecs, metric="manhattan"), k)
c_hammin = kmedoids.fasterpam(
    d(freq_vecs.toarray(), freq_vecs.toarray(), metric="hamming", n_jobs=-1), k
)


In [68]:
def show_clusters(labels, docs, n=40, save_to_file=None):
    """Prints text clustering

    Args:
        labels (list[Any]): list of cluster labels

        docs (list[str]): list of texts

        n (int, optional): Prints last n lines. Defaults to 40.

        save_to_file (Any, optional): If type==str saves output to file 'save_to_file'. Defaults to None.
    """

    cluster_dict = {label: [] for label in labels}
    for label, text in zip(labels, docs):
        cluster_dict[label].append(text)

    s = "==" * 42 + "\n"
    for label, texts in cluster_dict.items():
        s += "#" * 42 + "\n"
        for text in texts:
            s += text
        s += "\n"
        
    s += "==" * 42 + "\n"

    if type(save_to_file) == str:
        open(save_to_file, "w").write(s)

    print("\n".join(s.split(sep="\n")[-n:]))


In [69]:
show_clusters(c_euclid.labels, docs)


##########################################
ZHEJIANG JONWAYMACHINERY&ELECTRICMANUFACTURE CO LTD.DATANG INDUSTRIALZONE JIANTIAO TOWN SANMEN COUNTYTAIZHOU CITY
ZHEJIANG JONWAYMACHINERY&ELECTRICMANUFACTURE CO LTD.DATANG INDUSTRIALZONE JIANTIAO TOWN SANMEN COUNTYTAIZHOU CITY TEL:86-576-83431988

##########################################
ZHEJIANG JUNMA ALUMINIUM INDUSTRYCO.,LTD.  Add:No 8 Junma road, Huandong  street industrial park ZhujiZhejiang, China 311800

##########################################
ZHEJIANG NEWWAY IMPORT AND EXPORT CO.,LTD. NO 555 HAI YAN TANG ROAD JIAXING ZHEJIANG CHINA
ZHEJIANG TONGDE IMPORT AND EXPORTCO.,LTD. ROOM 1207-1208,JINMAO BUILDING NO. 699 CHOUZHOU NORTH ROAD ,ZHEJIANG CHINA
ZHEJIANG TONGDE IMPORT AND EXPORT CO.,LTD. ROOM 1207-1208,JINMAO BUILDING NO.699 CHOUZHOU NORTH ROAD,ZHEJIANG CHINA
ZHEJIANG YINMAO IMPORT&EXPORT CO.,LTD NO.777 HONGXING ROAD, JIAXING,ZHEJIANG,CHINA

##########################################
ZHEJIANG PROVINCIAL LIGHT&TEXTILEINDUSTRY GRO

In [70]:
show_clusters(c_manhat.labels, docs)


ZEN CONTINENTAL (TIANJIN) ENTERPRISES CO.,LTD.SHENZHEN BRANCH RM.D,12F, TIMES PLAZA, NO.1 TAIZI ROAD,SHEKOU, SHENZHEN,GUANGDONG, CHINA 0755-2681-8599 0755-2689-1817 ZIP:518067
ZEN CONTINENTAL (TIANJIN) ENTERPRISES CO.,LTD.SHENZHEN BRANCH RM.D,12F, TIMES PLAZA, NO.1 TAIZI ROAD, SHEKOU,SHENZHEN,GUANGDONG, CHINA 0755-2681-8599 0755-2689-1817 ZIP:518067

##########################################
ZHANGJIAGANG LOOP IMP.AND EXP CORP15E FUGANG BLDG.,RENMIN ROAD ZHANGJIAGANG CITY,JIANGSU PROVINCE,CN
ZHANGJIAGANG LOOP IMP.AND EXP CORP15E FUGANG BLDG., RENMIN ROAD ZHANGJIAGANG CITY,JIANGSU PROVINCE,CN

##########################################
ZHEJIANG AOTAI MACHINE MANUFACTURING CO.,LTD THE 10TH ROAD ,BINHAI THIRD AVNUE,BINHAI INDUSTRY AREA,WENZHOU,325025,CHINA CONTACT:PIER WANG TEL:+86-577-86830918 FAX:+86 577-868

##########################################
ZHEJIANG ARTS&CRAFTS IMP.&EXP. CO.,LTD. NO.12  ZHONGSHAN BEI ROAD,HANGZHOU, CHINA
ZHEJIANG HANGCHA IMP.&EXP.CO.,LTDNO.398 SHI QIAO ROAD,H

In [71]:
show_clusters(c_hammin.labels, docs)


##########################################
ZHEJIANG LAMDA BRAKE PADS CO.,LTD.NO.76 LUOSHAN ROAD,XIALIN INDUSTRIAL DISTRICT, RUIAN CITY,ZHEJIANG PROVINCE,CHINA

##########################################
ZHEJIANG LIWANG INDUSTRIAL AND  TRADING CO.,LTD O/B OF KONNER LIMITEDNO.226, RENMIN ROAD,NINGHAI,NINGBO,CHINA

##########################################
ZHEJIANG MAYANG INDUSTRIES CO.,LTDGUMASHAN INDUSTRY WANGZHAI TOWN WUYI CITY ZHEJIANG PROVINCE CHINA

##########################################
ZHEJIANG PROVINCIAL LIGHT&TEXTILEINDUSTRY GROUP IMP.&EXP.CO.,LTD.15/F,BLDG 8,UNITED PLAZA,NO.58 QIANJIANG ROAD,HANGZHOU,P.R.CHINA
ZHEJIANG PROVINCIAL LIGHT&TEXTILEINDUSTRY GROUP IMP.&EXP.CO.,LTD.15/F,BLDG 8,UNITED PLAZA,NO.58 QIANJIANG ROAD,HANGZHOU,P.R.CHINA

##########################################
ZHEJIANG SHENG PUJIANG WUSHI LOCKSCO.,LTD, YUEXI HUANGZHAI TOWN, PUJIANG CITY,ZHEJIANG PROVINCE, CHINA

##########################################
ZHEJIANG TONGDE IMPORT AND EXPORTCO.,LTD. ROOM 1

5. Porównaj jakość wyników sposobami zaimplementowanymi w pkt. 2.

In [33]:
lines = open("clusters.txt", "r").read().splitlines()
lines = list(filter(lambda text: text != "", lines))

model_docs, model_labels, label = [], [], 0
for line in lines:
    if line != "#" * 10:
        model_docs.append(line)
        model_labels.append(label)
    else:
        label += 1

X = CountVectorizer().fit_transform(model_docs)

In [34]:
print("Davies--Bouldin indexes")
print(
    "Model clusterization                ", 
    DB_idx(X.toarray(), labels=model_labels)
)
print(
    "k-medoids clusterization (euclidean)",
    DB_idx(freq_vecs.toarray(), c_euclid.labels),
)
print(
    "k-medoids clusterization (manhattan)",
    DB_idx(freq_vecs.toarray(), c_manhat.labels),
)
print(
    "k-medoids clusterization (hamming)  ",
    DB_idx(freq_vecs.toarray(), c_hammin.labels),
)


Davies--Bouldin indexes
Model clusterization                 1.112534158155742
k-medoids clusterization (euclidean) 1.5231276524933224
k-medoids clusterization (manhattan) 1.3434092302381833
k-medoids clusterization (hamming)   1.4500554434939745
